In [1]:
import numpy as np
from numpy import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os
print(os.path.abspath('.'))

D:\scBiG\reproducibility\datasets


In [3]:
from prosstt import simulation as sim
from prosstt import sim_utils as sut
from prosstt.tree import Tree

In [4]:
t = Tree(topology=[["0", "1"], ["1", "2"], ["2", "3"], ["3", "4"], ["4", "5"]],
         time={"0":100, "1":100, "2":100, "3":200, "4":100, "5":200},
         num_branches=6,
         branch_points=0,
         modules=15,
         G=10000,
         density=None,
         root="0")

In [5]:
##Return the start to end time of each branch
t.branch_times()

defaultdict(list,
            {'0': [0, 99],
             '1': [100, 199],
             '2': [200, 299],
             '3': [300, 499],
             '4': [500, 599],
             '5': [600, 799]})

In [6]:
# mya = np.min([0.05, 1 / t.modules])  
# mya
mya = np.min([0.075, 1 / t.modules])  
mya

0.06666666666666667

In [7]:
uMs, Ws, Hs = sim.simulate_lineage(t, a=mya, intra_branch_tol=-1, inter_branch_tol=0)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [8]:
num_cells=500
gene_scale = sut.simulate_base_gene_exp(t, uMs)
t.add_genes(uMs, gene_scale)
alpha = np.exp(random.normal(loc=np.log(0.1), scale=np.log(1), size=t.G))
beta = np.exp(random.normal(loc=np.log(1), scale=np.log(1.5), size=t.G)) + 1
series_points = [100, 200, 300, 400, 500]
point_std = [10, 10, 10, 10, 10] 
X, labs, brns, scalings = sim.sample_pseudotime_series(t, num_cells, series_points, point_std, alpha=alpha, beta=beta)

In [9]:
X

array([[ 0,  3,  0, ...,  0,  1,  0],
       [ 1,  6,  9, ..., 10, 16,  0],
       [ 0,  4,  2, ...,  4, 12,  2],
       ...,
       [ 1,  2,  6, ...,  3,  4,  0],
       [ 1,  5,  1, ...,  6,  5,  0],
       [ 1,  3,  5, ...,  8, 16,  0]])

In [10]:
 print("Sparsity: ", np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1]))

Sparsity:  0.5379522


In [11]:
import h5py
f = h5py.File('trajectory/DPT.h5','w') 
f['X'] = X 
f['Y'] = brns 
f['pt'] = labs
f.close()

#